## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-03-38-52 +0000,startribune,Joe Ryan starts Minnesota Twins toward 2-1 vic...,https://www.startribune.com/minnesota-twins-pi...
1,2025-07-12-01-58-41 +0000,latimes,Man sets California probation officer on fire ...,https://www.latimes.com/california/story/2025-...
2,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
3,2025-07-12-01-37-57 +0000,latimes,Federal judge temporarily halts alleged indisc...,https://www.latimes.com/california/story/2025-...
4,2025-07-12-01-17-02 +0000,latimes,Father of three Marines beaten by masked feder...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2300/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,immigration,6
176,trump,6
12,california,4
32,rate,4
95,canada,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
34,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,25
5,2025-07-12-01-09-58 +0000,latimes,California mayor warns residents amid immigrat...,https://www.latimes.com/california/story/2025-...,23
24,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,21
2,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...,20
47,2025-07-11-10-00-00 +0000,latimes,FBI investigating man who appeared to shoot at...,https://www.latimes.com/california/story/2025-...,18


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
34,25,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
5,23,2025-07-12-01-09-58 +0000,latimes,California mayor warns residents amid immigrat...,https://www.latimes.com/california/story/2025-...
2,20,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
44,18,2025-07-11-10-00-00 +0000,latimes,"Arellano: 'La migra, la migra': Inside Hunting...",https://www.latimes.com/california/story/2025-...
30,16,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
24,15,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
10,14,2025-07-11-23-55-01 +0000,startribune,Prosecutors argue ex-Sen. Justin Eichorn didn’...,https://www.startribune.com/prosecutors-argue-...
8,14,2025-07-12-00-16-13 +0000,latimes,SoCal surgery center staff confront ICE agents...,https://www.latimes.com/california/story/2025-...
7,13,2025-07-12-00-57-12 +0000,startribune,"New Wolves owners Marc Lore, Alex Rodriguez me...",https://www.startribune.com/marc-lore-alex-rod...
0,12,2025-07-12-03-38-52 +0000,startribune,Joe Ryan starts Minnesota Twins toward 2-1 vic...,https://www.startribune.com/minnesota-twins-pi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
